## Notebook 03 – Defining Volatility Regimes

This notebook defines and creates the volatility regimes for the universe (market) as well as the dispersion regimes across tickers, and exports both for downstream analysis.

### Step 0 - Import packages and functions

In [1]:
import sys, os
sys.path.append(os.path.abspath("../src"))
import pandas as pd
import numpy as np
from collections import OrderedDict

# Helper functions to compute regimes
from regime_definitions import compute_market_volatility_regime, compute_cross_sectional_dispersion_regime

C:\Users\edzz0\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


### Step 1 - Load Log Returns dataset

In [2]:
# Import log returns
log_returns = pd.read_parquet("../data/processed/log_returns.parquet")

### Step 2 - Compute volatility and dispersion regimes

In [3]:
methods = ["static", "rolling", "expanding"]
vol_regimes = OrderedDict()
disp_regimes = OrderedDict()

In [4]:
# Compute volatility regimes using 20-day rolling std of average market returns
# Compute dispersion regimes using 20-day rolling mean of cross-sectional std across tickers
# Uses default window parameter = 20 days, default lookback window for quantiles
# Compute all regimes
for method in methods:
    vol_regimes[method] = compute_market_volatility_regime(log_returns, quantile_method=method)
    disp_regimes[method] = compute_cross_sectional_dispersion_regime(log_returns, quantile_method=method)


### Step 3 - Print count of each volatility and dispersion regime.

In [5]:
# Print value counts
for method in methods:
    print(f"\nVolatility regime counts ({method}):")
    print(vol_regimes[method].value_counts(dropna=False))

for method in methods:
    print(f"\nDispersion regime counts ({method}):")
    print(disp_regimes[method].value_counts(dropna=False))


Volatility regime counts (static):
neutral     1556
high_vol    1153
low_vol     1153
dtype: int64

Volatility regime counts (rolling):
neutral     1620
low_vol     1280
high_vol     962
dtype: int64

Volatility regime counts (expanding):
neutral     1535
low_vol     1349
high_vol     978
dtype: int64

Dispersion regime counts (static):
neutral      1556
high_disp    1153
low_disp     1153
dtype: int64

Dispersion regime counts (rolling):
neutral      1577
low_disp     1191
high_disp    1094
dtype: int64

Dispersion regime counts (expanding):
high_disp    1429
neutral      1413
low_disp     1020
dtype: int64


### Step 4 - Export regime datasets

In [6]:
# Save regimes as parquet files
# Save parquet outputs
for method in methods:
    vol_regimes[method].to_frame(name="vol_regime").to_parquet(f"../data/processed/regime_market_vol_{method}.parquet")
    disp_regimes[method].to_frame(name="disp_regime").to_parquet(f"../data/processed/regime_cross_dispersion_{method}.parquet")